In [17]:
#edit distance 
def hammingDistance(x, y):
    nmm = 0
    for i in xrange(0, len(x)):
        if x[i] != y[i]:
            nmm += 1
    return nmm

In [18]:
#Programming homework week 3 Algorithms 

#First, download the provided excerpt of human chromosome 1
from Bio import SeqIO
for seq_record in SeqIO.parse("/Users/asudeberber/Documents/Coursera § Mycourses/GenomicDataScienceSpecializationJHU/Algorithms for DNA sequencing/chr1.GRCh38.excerpt.fasta", "fasta"):
    print(seq_record.id)
    print(repr(seq_record.seq))
    print(len(seq_record))

CM000663.2_excerpt
Seq('TTGAATGCTGAAATCAGCAGGTAATATATGATAATAGAGAAAGCTATCCCGAAG...AGG')
800000


In [19]:
#Second, parse it using the readGenome function we wrote before 

def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome
genome = readGenome('/Users/asudeberber/Documents/Coursera § Mycourses/GenomicDataScienceSpecializationJHU/Algorithms for DNA sequencing/chr1.GRCh38.excerpt.fasta')
genome[:100]

'TTGAATGCTGAAATCAGCAGGTAATATATGATAATAGAGAAAGCTATCCCGAAGGTGCATAGGTCAACAATACTTGAGCCTAACTCAGTAGATCCTAAAA'

In [27]:
#Third, adapt the editDistance function we saw in practical (copied below) to answer questions 1 and 2 below. Your function should take arguments p (pattern), t (text) and should return the edit distance of the match between P and T with the fewest edits.
def editDistance(x, y):
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = i
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return D[-1][-1]

In [28]:
#Question 1 What is the edit distance of the best match between pattern 
x = 'GCTGATCGATCGTACGGCTGATCGATCGTACG'
y = genome
# and the excerpt of human chromosome 1? 
print(editDistance(x, y))

799968


In [22]:
#Question2 
#What is the edit distance of the best match between pattern 
p = 'GATTTACCAGATTGAG'
t = genome
# and the excerpt of human chromosome 1? 
print(editDistance(p, t))

799984


In [23]:
#overlaps between pair of reads
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match


In [24]:
#finding all overlaps ? 
from itertools import permutations

def naive_overlap_map(reads, k):
    olaps = {}
    for a, b in permutations(reads, 2):
        olen = overlap(a, b, min_length=k)
        if olen > 0:
            olaps[(a, b)] = olen
    return olaps

In [25]:
#Question3 and 4
from Bio import SeqIO
for seq_record in SeqIO.parse("/Users/asudeberber/Documents/Coursera § Mycourses/GenomicDataScienceSpecializationJHU/Algorithms for DNA sequencing/ERR266411_1.for_asm.fastq", "fastq"):
    print(seq_record.id)
    print(repr(seq_record.seq))
    print(len(seq_record))



ERR266411.1
Seq('TAAACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGA...CTC')
100
ERR266411.2
Seq('AACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAA...CCC')
100
ERR266411.4
Seq('AACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAA...CTG')
100
ERR266411.5
Seq('AGCCGACGTTTTGGCGGCGCAACCTGTGACGACAAATCTGCTCAAATTTATGCG...ATC')
100
ERR266411.8
Seq('GACAAATCTGCTCAAATTTATGCGCGCTTCGATAAAAATGATTGGCGTATCCAA...TCG')
100
ERR266411.9
Seq('CTGTAGCCGACGTTTTGGCGGCGCAACCTGTGACGACAAATCTGCTCAAATTTA...TTT')
100
ERR266411.10
Seq('CTGTGACGACAAATCTGCTCAAATTTATGCGCGCTTCGATAAAAATGATTGGCG...AAC')
100
ERR266411.11
Seq('CAAATCTGCTCAAATTTATGCGCGCTTCGATAAAAATGATTGGCGTATCCAACC...GGA')
100
ERR266411.12
Seq('GTAAACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAG...ACC')
100
ERR266411.13
Seq('GTAAACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAG...ACT')
100
ERR266411.14
Seq('TTACTGTAGCCGACGTTTTGGCGGCGCAACCTGTGACGACAAATCTGCTCAAAT...GAG')
100
ERR266411.15
Seq('AAACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGA

In [26]:
# find all pairs of reads with an exact suffix/prefix match of length at least 30. Don't overlap a read with itself; if a read has a suffix/prefix match to itself, ignore that match.  Ignore reverse complements.


def find_suffix_prefix_matches(fastq_file):
    sequences = {}
    matches = []

    # Read the fastq file and store sequences in a dictionary
    for record in SeqIO.parse(fastq_file, "fastq"):
        sequences[record.id] = str(record.seq)

    # Iterate over each sequence
    for id1, seq1 in sequences.items():
        # Iterate over other sequences
        for id2, seq2 in sequences.items():
            # Check if the sequences are different and if one is a suffix/prefix of the other
            if id1 != id2 and (seq1.endswith(seq2[:30]) or seq1.startswith(seq2[-30:])):
                matches.append((id1, id2))

    return matches

# Example usage:
fastq_file = "/Users/asudeberber/Documents/Coursera § Mycourses/GenomicDataScienceSpecializationJHU/Algorithms for DNA sequencing/ERR266411_1.for_asm.fastq"
matches = find_suffix_prefix_matches(fastq_file)
print("Pairs of reads with an exact suffix/prefix match of length at least 30:")
for match in matches:
    print(match)



KeyboardInterrupt: 

In [ ]:
#number of matches equal to number of edges

from Bio import SeqIO
from collections import defaultdict
import time

def find_suffix_prefix_matches(fastq_file):
    start_time = time.time()

    # Dictionary to store sequences by their prefix/suffix
    seq_dict = defaultdict(list)

    # Iterate through the fastq file and store sequences by their prefix/suffix
    for record in SeqIO.parse(fastq_file, "fastq"):
        seq = str(record.seq)
        if len(seq) >= 30:
            seq_dict[seq[:30]].append(seq)

    # List to store pairs of reads with exact prefix/suffix match
    matches = []

    # Iterate through the sequences to find matches
    for prefix, seq_list in seq_dict.items():
        if len(seq_list) > 1:
            for i in range(len(seq_list)):
                for j in range(i+1, len(seq_list)):
                    if seq_list[i] != seq_list[j]:
                        matches.append((seq_list[i], seq_list[j]))

    end_time = time.time()
    print("Execution time:", end_time - start_time, "seconds")
    return matches

if __name__ == "__main__":
    fastq_file = "/Users/asudeberber/Documents/Coursera § Mycourses/GenomicDataScienceSpecializationJHU/Algorithms for DNA sequencing/ERR266411_1.for_asm.fastq"
    matches = find_suffix_prefix_matches(fastq_file)
    print("Number of matches:", len(matches))


Execution time: 0.4000871181488037 seconds
Number of matches: 29754


In [ ]:

matches = find_suffix_prefix_matches(fastq_file)
num_edges = len(matches)
print("Number of edges in the overlap graph:", num_edges)

Execution time: 0.27718210220336914 seconds
Number of edges in the overlap graph: 29754


In [ ]:
#To determine the number of nodes in the overlap graph that have at least one outgoing edge 
#(i.e., how many reads have a suffix involved in an overlap), we need to count the unique reads that are involved in at least one overlap.

matches = find_suffix_prefix_matches(fastq_file)

# Extract all reads involved in overlaps
reads_involved = set()
for match in matches:
    reads_involved.update(match)

num_nodes_with_outgoing_edge = len(reads_involved)
print("Number of nodes with at least one outgoing edge:", num_nodes_with_outgoing_edge)

Execution time: 0.4065430164337158 seconds
Number of nodes with at least one outgoing edge: 8224


In [31]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

def editDistance(x, y):
    """Returns the edit distance between two strings, x and y"""
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = i
    
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return D[-1][-1]

def bestApproximateMatchEditDistance(p, t):
    """Returns the edit distance between two strings, p and t"""
    # Create distance matrix
    D = []
    for i in range(len(p)+1):
        D.append([0]*(len(t)+1))
    
    # Initialize first row and column of matrix
    for i in range(len(p)+1):
        D[i][0] = i
    # See slide 4 on  0440_approx__editdist3.pdf
    # First row is already initialised to zero so we simply just comment the following two lines.
    #for i in range(len(p)+1):
    #    D[0][i] = i
    
    # Fill in the rest of the matrix
    for i in range(1, len(p)+1):
        for j in range(1, len(t)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if p[i-1] == t[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)

    # Best Approximate Match Distance is the smallest value of the last row
    return min(D[-1])

def test1():
    """P = GCGTATGC within T = TATTGGCTATACGGTT had 2 edits."""
    assert bestApproximateMatchEditDistance('GCGTATGC', 'TATTGGCTATACGGTT') == 2

def question1(t):
    """What is the edit distance of the best match between pattern GCTGATCGATCGTACG and the excerpt of human chromosome 1? (Don't consider reverse complements.)"""
    print("Question1: " + str(bestApproximateMatchEditDistance('GCTGATCGATCGTACG', t)))

def question2(t):
    """What is the edit distance of the best match between pattern GATTTACCAGATTGAG and the excerpt of human chromosome 1? (Don't consider reverse complements.)"""
    print("Question2: " + str(bestApproximateMatchEditDistance('GATTTACCAGATTGAG', t)))

def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching a prefix of 'b' that is at least 'min_length' characters long.  If no such overlap exists, return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

def overlap_all_pairs(reads, min_length):
    from pprint import pprint
    
    overlap_map = {}
    overlap_graph = {}
    overlap_pairs = []

    #We use a Python dictionary to associate each k-mer with its corresponding set. 
    suffixDict = {}
    for read in reads:
        kmers = getkmers(read, min_length)
        #print(kmers)
        #(1) For every k-mer in a read, we add the read to the set object corresponding to that k-mer.
        for kmer in kmers:
            if not kmer in suffixDict.keys():
                #Let every k-mer in the dataset have an associated Python set object, which starts out empty. 
                suffixDict[kmer] = set()
            suffixDict[kmer].add(read)
    #pprint(suffixDict)

    #(2) Now, for each read a, we find all overlaps involving a suffix of a
    for read in reads:
        #we take a's length-k suffix,
        suffix = read[-min_length:]
        #if len(suffix) < min_length:
        #    continue

        #find all reads containing that k-mer (obtained from the corresponding set) ...
        matching_reads = suffixDict[suffix]

        #...and call overlap(a, b, min_length=k) for each.
        for read2 in matching_reads:
            # The most important point is that we do not call overlap(a, b, min_length=k) if b does not contain the length-k suffix of a.
            #if read2.find(suffix) >= 0 and read2 != suffix :
            if read2 != read :
                val = overlap(read, read2, min_length) 
                if val > 0:
                    overlap_map[ (read, read2) ] = val
                    overlap_graph[read] = read2
                    overlap_pairs.append( (read,read2) )

    #pprint(overlap_map)
    #pprint(overlap_pairs)
    #pprint(overlap_graph)
    return overlap_pairs, overlap_map, overlap_graph



def getkmers(read, kmer_length):
    """our read is GATTA and k=3, we would add GATTA to the set objects for GAT, ATT and TTA."""
    return [ read[i:i+kmer_length] for i in range(len(read)+1-kmer_length) ]

def test2():
    kmers = getkmers('GATTA', 3)
    assert kmers == ['GAT', 'ATT', 'TTA']

def example1():
    reads = ['ABCDEFG', 'EFGHIJ', 'HIJABC']
    assert overlap_all_pairs(reads, 3)[0] == [('ABCDEFG', 'EFGHIJ'), ('EFGHIJ', 'HIJABC'), ('HIJABC', 'ABCDEFG')]
    assert overlap_all_pairs(reads, 4)[0] == []

def example2():
    from pprint import pprint

    reads = ['CGTACG', 'TACGTA', 'GTACGT', 'ACGTAC', 'GTACGA', 'TACGAT']

    results4, overlap_map4, overlap_graph4 = overlap_all_pairs(reads, 4)
    expected4 = [('CGTACG', 'TACGTA'),
                 ('CGTACG', 'GTACGT'),
                 ('CGTACG', 'GTACGA'),
                 ('CGTACG', 'TACGAT'),
                 ('TACGTA', 'ACGTAC'),
                 ('TACGTA', 'CGTACG'),
                 ('GTACGT', 'TACGTA'),
                 ('GTACGT', 'ACGTAC'),
                 ('ACGTAC', 'GTACGA'),
                 ('ACGTAC', 'GTACGT'),
                 ('ACGTAC', 'CGTACG'),
                 ('GTACGA', 'TACGAT')]

    assert sorted(results4) ==  sorted(expected4) , "example2, first assert failed"

    results5, overlap_map5, overlap_graph5 = overlap_all_pairs(reads, 5)
    expected5 = [('CGTACG', 'GTACGT'),
                 ('CGTACG', 'GTACGA'),
                 ('TACGTA', 'ACGTAC'),
                 ('GTACGT', 'TACGTA'),
                 ('ACGTAC', 'CGTACG'),
                 ('GTACGA', 'TACGAT')] 

    assert sorted(results5) ==  sorted(expected5), "example2, second assert failed"

def question3and4(reads):
    from pprint import pprint

    overlap_pairs, overlap_map, overlap_graph = overlap_all_pairs(reads, 30)

    """Picture the overlap graph corresponding to the overlaps just calculated. How many edges are in the graph? 
    In other words, how many distinct pairs of reads overlap?"""
    print('Question3: ')
    print(len(overlap_map))

    """Picture the overlap graph corresponding to the overlaps computed for the previous question. 
    How many nodes in this graph have at least one outgoing edge? (In other words, how many reads have a suffix involved in an overlap?)"""
    print('Question4: ')
    print(len(overlap_graph))


    

def main():
    test1()
    print("All tests completed successfully")
    genome = readGenome('/Users/asudeberber/Documents/Coursera § Mycourses/GenomicDataScienceSpecializationJHU/Algorithms for DNA sequencing/chr1.GRCh38.excerpt.fasta')
    question1(genome)
    question2(genome)

    test2()
    example1()
    example2()
    print("All example tests completed successfully")

    reads, qualities = readFastq('/Users/asudeberber/Documents/Coursera § Mycourses/GenomicDataScienceSpecializationJHU/Algorithms for DNA sequencing/ERR266411_1.for_asm.fastq')
    question3and4(reads)
    print("All done")
    



if __name__ == "__main__":
    main()


All tests completed successfully
Question1: 3
Question2: 2
All example tests completed successfully
Question3: 
904746
Question4: 
7161
All done
